/home/fukuda/Stock Tweets for Sentiment Analysis and Prediction から新たにpicklesを作るにあたり，去年の手法を見返しつつ，今後同様の操作を行う際にわかりやすくするため，このフォルダで一通り手順をわかるようにメモすることにする　

/home/fukuda/Stock Tweets for Sentiment Analysis and Predictionからツイート関連のデータを取るのは新たなコードが若干必要，このデータセットの企業の10-Qを取る作業は去年のコードの使い回しが可能かと

In [2]:
import pickle
import re
import os
import pandas as pd
from datetime import datetime, timedelta

In [1]:
from edgartools import Company

# ティッカーシンボルを使用して企業情報を初期化
aapl = Company("AAPL")

# 10-K提出書類のリストを取得
ten_k_filings = aapl.get_filings(form_type='10-K')

print("Latest 10-K Filing:")
if ten_k_filings:
    latest_10k = ten_k_filings[0]
    print(f"- Accession Number: {latest_10k.accession_number}")
    print(f"- Filed At: {latest_10k.filed_at}")
    
    # 提出書類のHTML全体を取得する例
    # content = latest_10k.html_content 
    # print(content[:500]) # 最初の500文字を表示

ModuleNotFoundError: No module named 'edgartools'

In [4]:
# https://www.kaggle.com/datasets/equinxx/stock-tweets-for-sentiment-analysis-and-prediction
df = pd.read_csv("../../datasets/Stock Tweets for Sentiment Analysis and Prediction/archive/stock_tweets.csv")

In [5]:
# print(df)
unique_stock_names = df['Stock Name'].unique()
print(unique_stock_names)
print(len(unique_stock_names)) #25
column_names = df.columns
print(column_names)
unique_company_names = df['Company Name'].unique()
print(unique_company_names)
print(len(unique_company_names))
print(df.iloc[0])

['TSLA' 'MSFT' 'PG' 'META' 'AMZN' 'GOOG' 'AMD' 'AAPL' 'NFLX' 'TSM' 'KO'
 'F' 'COST' 'DIS' 'VZ' 'CRM' 'INTC' 'BA' 'BX' 'NOC' 'PYPL' 'ENPH' 'NIO'
 'ZS' 'XPEV']
25
Index(['Date', 'Tweet', 'Stock Name', 'Company Name'], dtype='object')
['Tesla, Inc.' 'Microsoft Corporation' 'Procter & Gamble Company'
 'Meta Platforms, Inc.' 'Amazon.com, Inc.' 'Alphabet Inc.'
 'Advanced Micro Devices, Inc.' 'Apple Inc.' 'Netflix, Inc.'
 'Taiwan Semiconductor Manufacturing Company Limited'
 'The Coca-Cola Company' 'Ford Motor Company'
 'Costco Wholesale Corporation' 'The Walt Disney Company'
 'Verizon Communications Inc.' 'Salesforce, Inc.' 'Intel Corporation'
 'The Boeing Company' 'Blackstone Inc.' 'Northrop Grumman Corporation'
 'PayPal Holdings, Inc.' 'Enphase Energy, Inc.' 'NIO Inc.' 'Zscaler, Inc.'
 'XPeng Inc.']
25
Date                                    2022-09-29 23:41:16+00:00
Tweet           Mainstream media has done an amazing job at br...
Stock Name                                                

In [6]:
unique_stock_names = df['Stock Name'].unique()
for unique_stock_name in unique_stock_names:
    print(unique_stock_name)
    stock_rows = df[df['Stock Name'] == unique_stock_name]
    for i in range(3):
        print(stock_rows.iloc[i]["Tweet"])

TSLA
Mainstream media has done an amazing job at brainwashing people. Today at work, we were asked what companies we believe in &amp; I said @Tesla because they make the safest cars &amp; EVERYONE disagreed with me because they heard“they catch on fire &amp; the batteries cost 20k to replace”
Tesla delivery estimates are at around 364k from the analysts. $tsla
3/ Even if I include 63.0M unvested RSUs as of 6/30, additional equity needed for the RSUs is 63.0M x $54.20 = $3.4B. If the deal closed tomorrow at $54.20, Elon would need $2.0B for existing shares plus $3.4B for RSUs, so $5.4B new equity. $twtr $tsla
MSFT
A group of lawmakers led by Sen. Elizabeth Warren (D-Mass.) is asking the Federal Trade Commission to reject Amazon's, $AMZN, proposed acquisition of iRobot, per a letter shared first with Axios.
$NIO just because I'm down money doesn't mean this is a bad investment. The whole market, everything sucks right now. 2-5 years from now, I'm confident it will pay off. Long the best 

In [14]:
filtered_df = df[df['Tweet'].str.contains("Top 10 \$QQQ Holdings", case=False, na=False)]
print(filtered_df)
print(filtered_df.iloc[0]["Tweet"])

                            Date  \
56     2022-09-29 15:10:30+00:00   
37426  2022-09-29 15:10:30+00:00   
41515  2022-09-29 15:10:30+00:00   
48355  2022-09-29 15:10:30+00:00   
52440  2022-09-29 15:10:30+00:00   
55976  2022-09-29 15:10:30+00:00   
74118  2022-09-29 15:10:30+00:00   

                                                   Tweet Stock Name  \
56     Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...       TSLA   
37426  Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...       MSFT   
41515  Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...         PG   
48355  Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...       AMZN   
52440  Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...       GOOG   
55976  Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...       AAPL   
74118  Top 10 $QQQ Holdings \n\nAnd Credit Rating\n\n...       COST   

                       Company Name  
56                      Tesla, Inc.  
37426         Microsoft Corporation  
41515      Procter & Gamble 

In [ ]:
df['date'] = pd.to_datetime(df['date'])
filtered_df = df[(df['date'].dt.date() == pd.to_datetime('2022-09-29').date()) & (df['Stock Name'] == 'TSLA')]
print(filtered_df)

In [8]:
unique_stock_names = df['Stock Name'].unique()
stock_names_longer_1825 = []
stock_names_longer_1095 = []

for unique_stock_name in unique_stock_names:
    stock_rows = df[df['Stock Name'] == unique_stock_name]
    # print(unique_stock_name, len(stock_rows)) #各企業ごとのツイート数　かなりばらつきがある　ex TSLA 37422 KO 310
    #１年間（30-09-2021 till 30-09-2022）のデータセットで1日あたり５件ツイートが必要とすると，１８２５以上ツイートあるものが対象
    print(unique_stock_name, len(stock_rows))
    if len(stock_rows) > 1825:
        stock_names_longer_1825.append([unique_stock_name, len(stock_rows)])
    if len(stock_rows) > 1095: #1日あたり3件ツイート
        stock_names_longer_1095.append([unique_stock_name, len(stock_rows)])
    # print(unique_stock_name, stock_rows.iloc[0]["Tweet"])
    # print("")

print(stock_names_longer_1825)
print(len(stock_names_longer_1825)) #9

print(stock_names_longer_1095)
print(len(stock_names_longer_1095)) #11

#先生曰くツイート少ない企業でも，ツイートなしで機能するという点で強調できるから，pickleに追加してもいいとか

TSLA 37422
MSFT 4089
PG 4089
META 2751
AMZN 4089
GOOG 1291
AMD 2227
AAPL 5056
NFLX 1727
TSM 11034
KO 310
F 31
COST 393
DIS 635
VZ 123
CRM 233
INTC 315
BA 399
BX 50
NOC 31
PYPL 843
ENPH 216
NIO 3021
ZS 193
XPEV 225
[['TSLA', 37422], ['MSFT', 4089], ['PG', 4089], ['META', 2751], ['AMZN', 4089], ['AMD', 2227], ['AAPL', 5056], ['TSM', 11034], ['NIO', 3021]]
9
[['TSLA', 37422], ['MSFT', 4089], ['PG', 4089], ['META', 2751], ['AMZN', 4089], ['GOOG', 1291], ['AMD', 2227], ['AAPL', 5056], ['NFLX', 1727], ['TSM', 11034], ['NIO', 3021]]
11
